# Webscraping at Zolo Real State Website.

### Getting the links to each building at Zolo website.

In [226]:
"""
 Second way --- using the "webdriver-manager"
 we will be able to always download the latest version of the "chromedriver" 
 using our Google Chrome version as a reference.
"""

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from time import sleep

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

#allow us to navigate trough any page and to open a html archive in our pc.
#driver.get(r"https://www.zolo.ca/")
list_links=[]

for page in range(1,5):
    #define the link of the current page to be opened.
    current_page = 'https://www.zolo.ca/toronto-real-estate/apartments-for-rent/page-'+str(page)
    #open a page based on the link formed.
    driver.get(current_page)
    #print(current_page)
    #gets all the articles to a list.
    articles = driver.find_elements(By.TAG_NAME, 'article')
    #inside of each article it is looking for the "a hef" where we got the link to the detailed page.
    for article in articles:
        try:
            link = article.find_element(By.TAG_NAME, 'a').get_attribute('href')
            list_links.append(link)
            #print(link)
        except:
            print('erro')
    #best order of elements -> ID > XPATH > CLASS_NAME
    #sleep(1)
    
    if list_links == 19999:
        break
        
#creating the dataframe to input into sheets.
df_links = pd.DataFrame(columns=['links_zolo'], data=list_links)

#removing link duplicates.
df_links.drop_duplicates(inplace=True)


driver.quit()

https://www.zolo.ca/toronto-real-estate/426-university-avenue/1003
https://www.zolo.ca/toronto-real-estate/21-lawren-harris-square/901
https://www.zolo.ca/toronto-real-estate/125-village-green-square/506
https://www.zolo.ca/toronto-real-estate/275-village-green-square/2622
https://www.zolo.ca/toronto-real-estate/159-wellesley-street-east/2710
https://www.zolo.ca/toronto-real-estate/126-simcoe-street/2302
https://www.zolo.ca/toronto-real-estate/12-york-street/5509
https://www.zolo.ca/toronto-real-estate/18-harbour-street/906
https://www.zolo.ca/toronto-real-estate/188-fairview-mall-drive/727
https://www.zolo.ca/toronto-real-estate/65-st-mary-street/1808
https://www.zolo.ca/toronto-real-estate/32-trolley-crescent/304
https://www.zolo.ca/toronto-real-estate/85-east-liberty-street/809
https://www.zolo.ca/toronto-real-estate/33-university-avenue/503
https://www.zolo.ca/toronto-real-estate/8-trent-avenue-east/1210
https://www.zolo.ca/toronto-real-estate/77-mutual-street-north/ph-1
https://ww

### Sistematizando a montagem do dataframe

In [ ]:

#creating a service that will access Zolo website.
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

#creating the dataframe which we will input the data
df = pd.DataFrame(columns=['link', 'price', 'address', 'city', 'neighborhood', 'bed', 'bath', 'sale_or_rent', 'size', 'add_date',
                          'update_date', 'mls_code', 'style'])

for i, link in enumerate(df_links['links_zolo']):
    sleep(0.2)
    try:
        driver.get(link)
        
        try:
            price = driver.find_element(By.XPATH, '/html/body/section[2]/div/div/section[1]/section[2]/div[1]').text
        except:
            price = 'NA'

        try:
            address = driver.find_element(By.XPATH, '/html/body/section[2]/div/div/section[1]/section[1]/h1').text
        except:
            address = 'NA'
    
        try:
            city = driver.find_element(By.XPATH, '/html/body/section[2]/div/div/section[1]/section[1]/div/a[1]').text
        except:
            city='NA'
    
        try:
            neighborhood = driver.find_element(By.XPATH, '/html/body/section[2]/div/div/section[1]/section[1]/div/a[2]').text
        except:
            neighborhood = 'NA'
        
        try:
            bed = driver.find_element(By.XPATH, '/html/body/section[2]/div/div/section[1]/section[1]/ul/li[1]').text
        except:
            bed = 'NA'

        try:
            bath = driver.find_element(By.XPATH, '/html/body/section[2]/div/div/section[1]/section[1]/ul/li[2]').text
        except:
            bath = 'NA'
    
        try:
            sale_or_rent = driver.find_element(By.XPATH, '/html/body/section[2]/div/div/section[1]/section[2]/div[2]/div[1]').text
        except:
            sale_or_rent = driver.find_element(By.XPATH, '/html/body/section[2]/div/div/section[1]/section[2]/div[3]/div[1]').text

        try:
            size = driver.find_element(By.XPATH, '/html/body/section[2]/div/div/section[5]/section/div[1]/dl[3]/dd/span').text
        except:
            size='NA'
            
        try:
            add_date = driver.find_element(By.XPATH, '/html/body/section[2]/div/div/section[5]/section[1]/div[2]/dl[1]/dd/span').text
        except:
            add_date = 'NA'

        try:
            update_date = driver.find_element(By.XPATH, '/html/body/section[2]/div/div/section[5]/section[1]/div[2]/dl[2]/dd/time/span').text
        except:
            update_date = 'NA'
            
        try:
            mls_code = driver.find_element(By.XPATH, '/html/body/section[2]/div/div/section[5]/section[1]/div[2]/dl[4]/dd/span').text
        except:
            mls_code = 'NA'
            
        try:
            style = driver.find_element(By.XPATH, '/html/body/section[2]/div/div/section[5]/section[1]/div[1]/dl[2]/dd/span').text
        except:
            style = 'NA'
            
        df.loc[link] = [link, price, address, city, neighborhood, bed, bath, sale_or_rent, size, add_date, update_date, mls_code, style]
    
    except:  #SE DER ERRO NO LINK DO SITE, ATENCAO AQUI
        print('erro')
        
    

    
    
driver.quit()

erro
erro


In [224]:
df.shape

(46, 13)

In [216]:
import gspread
import pandas as pd

CODE = '1b4DllN1xFdujCzYnaMMWyC4tBg_gdmzT6G9GkqGT9Jo'

gc = gspread.service_account(filename='key.json')

sh = gc.open_by_key(CODE)

ws = sh.worksheet('data_toronto_real_state')

ws.update([df.columns.values.tolist()] + df.values.tolist())

{'spreadsheetId': '1b4DllN1xFdujCzYnaMMWyC4tBg_gdmzT6G9GkqGT9Jo',
 'updatedRange': 'data_toronto_real_state!A1:L147',
 'updatedRows': 147,
 'updatedColumns': 12,
 'updatedCells': 1764}

In [123]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from time import sleep

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

#driver.get('https://www.zolo.ca/toronto-real-estate/120-varna-drive/612')

driver.get('https://www.zolo.ca/toronto-real-estate/50-thomas-riley-road/1106')


price = driver.find_element(By.XPATH, '/html/body/section[2]/div/div/section[1]/section[2]/div[1]').text
city = driver.find_element(By.XPATH, '/html/body/section[2]/div/div/section[1]/section[1]/div/a[1]').text
neighborhood = driver.find_element(By.XPATH, '/html/body/section[2]/div/div/section[1]/section[1]/div/a[2]').text
re_type = driver.find_element(By.XPATH, '/html/body/section[2]/div/div/section[1]/section[2]/div[2]/div[1]').text
bed = driver.find_element(By.XPATH, '/html/body/section[2]/div/div/section[1]/section[1]/ul/li[1]').text
bath = driver.find_element(By.XPATH, '/html/body/section[2]/div/div/section[1]/section[1]/ul/li[2]').text
type_ = driver.find_element(By.XPATH, '/html/body/section[2]/div/div/section[5]/section/div[1]/dl[1]/dd/span').text
style = driver.find_element(By.XPATH, '/html/body/section[2]/div/div/section[5]/section/div[1]/dl[2]/dd/span').text
size = driver.find_element(By.XPATH, '/html/body/section[2]/div/div/section[5]/section/div[1]/dl[3]/dd/span').text

print('price',price)
print('city',city)
print('neighborhood', neighborhood)
print('re_type', re_type)
print('bed', bed)
print('bath', bath)
print('type_', type_)
print('style', style)
print('size', size)

driver.quit()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="listing"]/div/div/section[1]/section[2]/div[3]/div[1]"}
  (Session info: chrome=112.0.5615.137)
Stacktrace:
0   chromedriver                        0x000000010098bae8 chromedriver + 4958952
1   chromedriver                        0x00000001009837b3 chromedriver + 4925363
2   chromedriver                        0x00000001005451d6 chromedriver + 475606
3   chromedriver                        0x0000000100588a20 chromedriver + 752160
4   chromedriver                        0x0000000100588c81 chromedriver + 752769
5   chromedriver                        0x00000001005ccf64 chromedriver + 1032036
6   chromedriver                        0x00000001005aec3d chromedriver + 908349
7   chromedriver                        0x00000001005ca25b chromedriver + 1020507
8   chromedriver                        0x00000001005ae9e3 chromedriver + 907747
9   chromedriver                        0x00